The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
#!/bin/bash

# checking if the folder students and the file exists, so:
if [[ ! -d "students" ]];
then
	echo "creating folder 'students'"
	mkdir students
	echo "downloading csv file"
	wget "https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv" -O ./students/student_list.csv
elif [[ ! -f "./students/student_list.csv" ]];
then
	echo "downloading csv file"
	wget "https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv" -O ./students/student_list.csv
else
	echo "folder and file already there."
fi

# making now 2 files: one contains only POD students, the other only Physics students, so:
grep "\(.*\)[Pp][Oo][Dd]\(.*\)$" ./students/student_list.csv > ./students/student_list_POD.csv
grep "\(.*\)[Pp][Hh][Yy][Ss][Ii][Cc][Ss]\(.*\)$" ./students/student_list.csv > ./students/student_list_PHYSICS.csv

# for each letter of the alphabet, count the number of students whose surname starts with that letter
if [[ -f "./students/counts.txt" ]];
then
	rm ./students/counts.txt
fi
for i in {A..Z};
do
	counts=$(sed -n "2,$ p" ./students/student_list.csv | grep -c "^${i}")
	echo "${counts} ${i}" >> ./students/counts.txt
done

# finding out the letters with the max freq
h_freq_letter=$(sort -nr ./students/counts.txt | head -1)
h_freq=$(echo ${h_freq_letter} | sed -n "s/ .*$//p")
h_letter=$(echo ${h_freq_letter} | sed -n "s/^\(.*\) //p")

# matching to get the char with this frequency
echo "The letter with the highest frequency is "$h_letter" F("$h_letter") = "$h_freq

# students modulo 18
N_students=$(grep -c "\(\([Pp][Oo][Dd]\).*$\)\|\(\([Pp][Hh][Yy][Ss][Ii][Cc][Ss]\).*$\)" ./students/student_list.csv)
i=0
while [[ $i -le $N_students ]];
do
	# checking if the group file exists and if it does, remove it
	if [[ $i -eq 0 ]];
	then
		for j in [1..18];
		do
			if [[ -f "./students/group_"${j}".txt" ]];
			then
				rm "./students/group_"${j}".txt"
			fi
		done
	fi
	
	# adding stuff to the groups
	group_idx=$(($i % 18))
	
	sed -n "$((i+2))p" "./students/student_list.csv" >> "./students/group_"$((group_idx+1))".txt"
	
	((i++))
done
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
#!/bin/bash

# removing metadata and commas
grep "^[^#]" data.csv | sed -n ":x s/\(, \)/ /; tx; p" > data.txt
grep "^[^#]" data.csv | sed -n ":x s/\(, \)/\n/; tx; p" > data_col.txt

# even number counting
N_rows=$(grep -c "." ./data_col.txt)
i=1
N_even=0
while [[ $i -le $N_rows ]];
do
	# checking
	n=$(sed -n "${i}p" ./data_col.txt)
	mod=$(((n % 2)))
	if [[ $mod -eq 0 ]];
	then
		((N_even++))
	fi
	((i++))
done
echo "There are "$N_even" even numbers."

# summing the first three coordinates
i=1
N_greater=0
N_less=0
while [[ $i -le $N_rows ]];
do
	# i increments by 6 each time
	squared_sum=0
	j=${i}
	jupper=$((j+3))
	while [[ $j -lt jupper ]];
	do
		# scanning each value
		value=$(sed -n "${j}p" ./data_col.txt)
		squared_sum=$(((squared_sum + value**2)))
		((j++))
	done
	# checking the squared sum using bc command	
	test=$(echo "sqrt(${squared_sum}) < (100*sqrt(3)/2)" | bc)
	if [[ test -eq 0 ]];
	then
		((N_less++))
	else
		((N_greater++))
	fi
	i=$(((i+6)))
done
echo "Sum of squares greater than limit: "${N_greater}
echo "Sum of squares smaller than limit: "${N_less}

# make n copies of data.txt. The i-th copy of data has all numbers divided by i
if [[ ! $1 -ge 1 ]];
then
	echo "ERROR: first argument must be greater or equal to 1."
	exit
fi

i=1
while [[ $i -le $1 ]];
do
	# for each i, we need to divide every number, so
	if [[ -f "./data_"${i}".txt" ]];
	then
		rm "./data_"${i}".txt"
	fi
	# proceding
	j=1
	while [[ $j -le $N_rows ]];
	do
		k=${j}
		kupper=$((k+6))
		line=""
		while [[ $k -lt $kupper ]];
		do
			# for each line in the file, must divide
			value=$(sed -n "${k}p" ./data_col.txt)
			divided=$(echo "${value} / ${i}" | bc)
			line=${line}" "${divided}
			# incrementing k
			((k++))
		done
		# saving the line
		line=${line}
		echo ${line} >> "./data_"${i}".txt"
		# incrementing by 6
		j=$(((j+6)))
	done
	echo "done writing ./data_"${i}".txt"
	((i++))
done
```